In [8]:
import geopandas
import pandas as pd
from math import ceil
from balancer import MasterPlan

In [9]:
Hectare = 10000

In [10]:
school = geopandas.read_file("../data/school.gepjson")
kindergarten = geopandas.read_file("../data/kindergarten.gepjson")
greening = geopandas.read_file("../data/greening.gepjson")

In [11]:
gdf = geopandas.read_parquet("../data/BLOCKS_AND_BUILDINGS_INFO.parquet")

In [12]:
gdf["current_building_area"] = gdf["current_living_area"] + gdf["current_industrial_area"]

In [13]:
gdf_ = gdf[
    [
        "block_id",
        "area",
        "current_living_area",
        "current_industrial_area",
        "current_population",
        "current_green_area",
        "floors",
    ]
]

In [15]:
gdf_ = (
    gdf_.merge(school[["id", "population_unprov_Школа"]], left_on="block_id", right_on="id")
    .merge(kindergarten[["id", "population_unprov_Детский сад"]], left_on="block_id", right_on="id")
    .merge(greening[["id", "population_unprov_Рекреационная зона"]], left_on="block_id", right_on="id")
)

In [16]:
gdf_.drop(["id_x", "id_y", "id"], axis=1, inplace=True)

In [17]:
gdf_["area"] = gdf_["area"] / Hectare
gdf_["current_living_area"] = gdf_["current_living_area"] / Hectare
gdf_["current_industrial_area"] = gdf_["current_industrial_area"] / Hectare
gdf_["current_green_area"] = gdf_["current_green_area"] / Hectare

In [18]:
sample = gdf_.sample()
sample = sample.to_dict("records")

In [19]:
mp = MasterPlan(
    area=sample[0]["area"],
    current_living_area=sample[0]["current_living_area"],
    current_industrial_area=sample[0]["current_industrial_area"],
    current_population=sample[0]["current_population"],
    current_green_area=sample[0]["current_green_area"],
    current_unprov_schoolkids=sample[0]["population_unprov_Школа"],
    current_unprov_kids=sample[0]["population_unprov_Детский сад"],
    current_unprov_green_population=sample[0]["population_unprov_Рекреационная зона"],
)

In [20]:
mp.func()

In [24]:
mp.results[["fun", "x"]]

,fun,x
0,4.065605,"[612.8369543413284, 0.0029999177035257975, 0.0..."
1,4.065626,"[612.8369071743484, 0.0029999007077349234, 0.0..."
2,4.065504,"[612.8369754249721, 0.003, 0.0009]"
3,4.065504,"[612.8369043216663, 0.003, 0.0009]"
4,4.065504,"[612.836944294888, 0.003, 0.0012]"


In [22]:
mp.optimal_values(*mp.select_optimal())

{'area': 8.65810410856975,
 'population': 624.0,
 'b': 30.0,
 'green_coef': 9.0,
 'living_area': 1.839,
 'sc_area': 0,
 'kg_area': 0,
 'green_area': 0.5517,
 'op_area': 0.0018390000000000001,
 'parking1_area': 0.463428,
 'parking2_area': 0.450555}

In [23]:
sample[0]

{'block_id': 3087,
 'area': 8.65810410856975,
 'current_living_area': 0.0,
 'current_industrial_area': 1.2519306399999999,
 'current_population': 11.0,
 'current_green_area': 0.0,
 'floors': 2.0,
 'population_unprov_Школа': 0,
 'population_unprov_Детский сад': 0,
 'population_unprov_Рекреационная зона': 0}